## Generate Geoparquet Data Samples

1. Convert ATL03 samples (whole granules) to geoparquet
2. Use SlideRule with the geoparquet export option to download granules by ID
3. Save to common bucket

In [9]:
import os
os.environ['USE_PYGEOS'] = '0'
from sliderule import icesat2, earthdata, io

In [10]:

#figure out the format sliderule expects granule IDs to be in
# find granules for a spatial and temporal query
box_lon = [-105, -105, -100, -100, -105]
box_lat = [-75, -77.5, -77.5, -75, -75]
poly = io.to_region(box_lon, box_lat)
resources = earthdata.cmr(short_name='ATL03', polygon=poly, time_start='2018-10-19', time_end='2018-10-20') 
granule = resources[0]

granule

'ATL03_20181019224323_03250112_005_01.h5'

In [12]:
!aws s3 ls s3://nasa-cryo-scratch/h5cloud/original/

2023-08-08 23:45:34 7754735138 ATL03_20181120182818_08110112_006_02.h5
2023-08-08 23:47:04 6997123664 ATL03_20190219140808_08110212_006_02.h5
2023-08-08 23:47:04 6925710500 ATL03_20200217204710_08110612_006_01.h5
2023-08-08 23:47:04 8392279594 ATL03_20211114142614_08111312_006_01.h5
2023-08-08 23:47:04 7954039827 ATL03_20230211164520_08111812_006_01.h5


In [4]:
# initialize
icesat2.init("slideruleearth.io")

In [ ]:
granules = ['ATL03_20181120182818_08110112_006_02.h5',
            'ATL03_20190219140808_08110212_006_02.h5',
            'ATL03_20200217204710_08110612_006_01.h5',
            'ATL03_20211114142614_08111312_006_01.h5',
            'ATL03_20230211164520_08111812_006_01.h5'] 

In [22]:
def get_gpq(granule):
    #granule = "ATL03_20181120182818_08110112_006_02.h5"
    asset = "icesat2"
    #output = f"s3://nasa-cryo-scratch/h5cloud/geoparquet/{granule}.gpq"
    output = f"~/{granule}.gpq"
    params = {
        "output" : {
            "path" : output,
            "format" : "parquet",
            "open_on_complete" : False,
        }
    }
    status = icesat2.atl03sp(params, granule, asset=asset)
    
    return status

In [20]:
%%time
status = icesat2.atl03s(params, granule, asset=asset) 

CPU times: user 53.7 ms, sys: 4.1 ms, total: 57.8 ms
Wall time: 3min 58s


In [ ]:
%%time
for granule in granules:
    result = get_gpq(granule)
    print(result)

In [21]:
status

's3://nasa-cryo-scratch/h5cloud/geoparquet/ATL03_20181120182818_08110112_006_02.h5.gpq'